# Support Vector Machines (SVM) - RBF Kernel

In [1]:
import sys
sys.path.insert(1, '../../libs')
from utils import get_data, temporal_train_test_split
from autorship import AuthorClassifier
from sklearn.preprocessing import MaxAbsScaler
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from nltk.tokenize import TweetTokenizer
import numpy as np
import pandas as pd

In [2]:
data = get_data("../../data/authors.csv")

In [3]:
tt = TweetTokenizer()

vectorizers = [CountVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                CountVectorizer(ngram_range=(1,5), analyzer="char"),
                CountVectorizer(ngram_range=(4,5), analyzer="char"),
                CountVectorizer(ngram_range=(3,8), analyzer="char"),
                TfidfVectorizer(ngram_range=(1,1), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,3), analyzer="word", tokenizer=tt.tokenize), 
                TfidfVectorizer(ngram_range=(1,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(4,5), analyzer="char"),
                TfidfVectorizer(ngram_range=(3,8), analyzer="char")]

In [4]:
from joblib import Parallel, delayed
def process(vectorizer):
    evaluation = list()
    usernames = list(np.unique(data["username"]))
    
    vectorizer_str = vectorizer.__str__()
    print(f"Running => {vectorizer_str}")

    for i in range(len(usernames)):
        author1 = usernames.pop()

        for author2 in usernames:
            X_train, X_test, y_train, y_test = temporal_train_test_split(   
                data, author1, author2)

            clf = AuthorClassifier(vectorizer, clf=SVC(random_state=42), scaler=MaxAbsScaler())
            clf.fit(X_train, y_train)
            y_pred = clf.predict(X_test)
            evaluation.append(clf.evaluate(y_test, y_pred))
            metrics = pd.DataFrame(evaluation)[["f1_macro", "recall_macro", "precision_macro", "accuracy", "auc_score"]].mean()
    
    return (vectorizer_str, metrics)
    
results = Parallel(n_jobs=6)(delayed(process)(vectorizer) for vectorizer in vectorizers)
print(results) 

Running => CountVectorizer(analyzer='char', ngram_range=(3, 8))
Running => CountVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f84fb211c70>>)
Running => CountVectorizer(analyzer='char', ngram_range=(1, 5))
Running => CountVectorizer(analyzer='char', ngram_range=(4, 5))
Running => CountVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7f11ca9e7c70>>)
Running => TfidfVectorizer(tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fb7190c3c70>>)


/home/vinicius/.local/share/virtualenvs/reddit-in-portuguese-CNZVnMpw/lib/python3.8/site-packages/sklearn/svm/_base.py:1206: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Running => TfidfVectorizer(ngram_range=(1, 3),
                tokenizer=<bound method TweetTokenizer.tokenize of <nltk.tokenize.casual.TweetTokenizer object at 0x7fb7190c3e20>>)
Running => TfidfVectorizer(analyzer='char', ngram_range=(1, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(4, 5))
Running => TfidfVectorizer(analyzer='char', ngram_range=(3, 8))


In [ ]:
metrics_df = pd.DataFrame([results[i][1] for i in range(len(results))])
metrics_df["vectorizer"] = [results[i][0] for i in range(len(results))]
metrics_df.to_csv("../../results/linear_rbf.csv")
metrics_df

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.846351,0.847796,0.854371,0.847127,0.921445,CountVectorizer(tokenizer=<bound method TweetT...
1,0.849517,0.851003,0.857799,0.850310,0.924921,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.832508,0.833614,0.838241,0.833045,0.911133,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.786555,0.788751,0.796968,0.788040,0.872845,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.802636,0.804326,0.811221,0.803705,0.889356,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.857344,0.857802,0.860218,0.857628,0.928573,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.853617,0.854035,0.857906,0.854209,0.929903,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.861566,0.861734,0.862389,0.861691,0.929556,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.805552,0.806158,0.807864,0.805922,0.885960,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.817224,0.817422,0.818528,0.817508,0.897528,"TfidfVectorizer(analyzer='char', ngram_range=(..."


Row with max sum of columns (remember that by the result of f1-score we use the precision and recall score)

In [ ]:
print(metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()])
print("\nVectorizer:", metrics_df.loc[metrics_df[['f1_macro', 'accuracy', 'auc_score']].sum(1).idxmax()]["vectorizer"])

f1_macro                                                    0.861566
recall_macro                                                0.861734
precision_macro                                             0.862389
accuracy                                                    0.861691
auc_score                                                   0.929556
vectorizer         TfidfVectorizer(analyzer='char', ngram_range=(...
Name: 7, dtype: object

Vectorizer: TfidfVectorizer(analyzer='char', ngram_range=(1, 5))


In [ ]:
pd.read_csv("../../results/linear_rbf.csv").drop("Unnamed: 0", axis=1)

,f1_macro,recall_macro,precision_macro,accuracy,auc_score,vectorizer
0,0.846351,0.847796,0.854371,0.847127,0.921445,CountVectorizer(tokenizer=<bound method TweetT...
1,0.849517,0.851003,0.857799,0.850310,0.924921,"CountVectorizer(ngram_range=(1, 3),\n ..."
2,0.832508,0.833614,0.838241,0.833045,0.911133,"CountVectorizer(analyzer='char', ngram_range=(..."
3,0.786555,0.788751,0.796968,0.788040,0.872845,"CountVectorizer(analyzer='char', ngram_range=(..."
4,0.802636,0.804326,0.811221,0.803705,0.889356,"CountVectorizer(analyzer='char', ngram_range=(..."
5,0.857344,0.857802,0.860218,0.857628,0.928573,TfidfVectorizer(tokenizer=<bound method TweetT...
6,0.853617,0.854035,0.857906,0.854209,0.929903,"TfidfVectorizer(ngram_range=(1, 3),\n ..."
7,0.861566,0.861734,0.862389,0.861691,0.929556,"TfidfVectorizer(analyzer='char', ngram_range=(..."
8,0.805552,0.806158,0.807864,0.805922,0.885960,"TfidfVectorizer(analyzer='char', ngram_range=(..."
9,0.817224,0.817422,0.818528,0.817508,0.897528,"TfidfVectorizer(analyzer='char', ngram_range=(..."
